# One-Time Signatures

## Syntax & Correctness

:::{note} One-Time Signature
:label: def-one-time-signature

A one-time signature scheme $\mathsf{OTS}$ is a tuple of polynomial-time algorithms $(\mathsf{Setup}, \mathsf{KeyGen}, \mathsf{Sign}, \mathsf{Verify})$ where:

- $\mathsf{Setup}(1^\lambda) \rightarrow \mathit{pp}$ is a probabilistic algorithm that takes the security parameter as input and outputs public parameters $\mathit{pp}$ including the message space $\mathcal{M}$.
- $\mathsf{KeyGen}(\mathit{pp}) \rightarrow (\mathit{sk}, \mathit{pk})$ is a probabilistic algorithm that takes public parameters $\mathit{pp}$ as input and outputs a secret key $\mathit{sk}$ and a public key $\mathit{pk}$.
- $\mathsf{Sign}(\mathit{pp}, \mathit{sk}, m) \rightarrow \sigma$ is a probabilistic algorithm that takes public parameters $\mathit{pp}$, a secret key $\mathit{sk}$, and a message $m \in \mathcal{M}$ as input and outputs a signature $\sigma$.
- $\mathsf{Verify}(\mathit{pp}, \mathit{pk}, m, \sigma) \rightarrow \{0, 1\}$ is a deterministic algorithm that takes public parameters $\mathit{pp}$, a public key $\mathit{pk}$, a message $m \in \mathcal{M}$, and a signature $\sigma$ as input and outputs $1$ (accept) if the signature is valid, and outputs $0$ (reject) otherwise.

:::

:::{note} Correctness
:label: def-ots-correctness

A one-time signature scheme $\mathsf{OTS} = (\mathsf{Setup}, \mathsf{KeyGen}, \mathsf{Sign}, \mathsf{Verify})$ is **correct** if for all $\lambda \in \mathbb{N}$, all $\mathit{pp} \in \mathsf{Setup}(1^\lambda)$, all $(\mathit{sk}, \mathit{pk}) \in \mathsf{KeyGen}(\mathit{pp})$, and all messages $m \in \mathcal{M}$:

$$
\Pr[\mathsf{Verify}(\mathit{pp}, \mathit{pk}, m, \mathsf{Sign}(\mathit{pp}, \mathit{sk}, m)) = 1] = 1
$$

where the probability is taken over the randomness of $\mathsf{Sign}$.

:::

## Security

:::{note} EUF-CMA Security
:label: def-euf-cma-ot

A one-time signature scheme $\mathsf{OTS}$ is **EUF-CMA secure** (Existential Unforgeability under Chosen Message Attack) if for all PPT adversaries $\mathcal{A}$, it holds that

$$
\mathcal{A}antage{EUF-CMA}{\mathcal{A}, \mathsf{OTS}} := \Pr[\mathsf{Game}~\mathsf{EUF-CMA}_{\mathsf{OTS}} = 1] = \mathrm{negl}.
$$

:::

:::{tip} Assert notation

Given a predicate $P$, we use "$\mathbf{assert} P$" as a shorthand for "$\mathbf{if} \neg P \mathbf{then} \mathbf{return} 0$" in the main procedure or "$\mathbf{if} \neg P \mathbf{then} \mathbf{return} \bot$" in an oracle procedure.

:::

:::{admonition} The EUF-CMA security game for a one-time signature scheme
:class: dropdown

```
Game EUF-CMA(OTS):
  params ← Setup(1^λ)
  (sk, pk) ← KeyGen(params)
  Q := ∅
  (m*, σ) ← A^Sign(params, pk)
  return m* ∉ Q ∧ Verify(params, pk, m*, σ) = 1

Oracle Sign(m):
  assert Q = ∅
  Q := {m}
  return Sign(params, sk, m)
```

:::

## Lamport Signatures

:::{admonition} The one-time Lamport signature scheme $\mathsf{LS}[\mathsf{H}, k]$
:class: dropdown

```
Setup(1^λ):
  params := H.Gen(1^λ)
  return params

KeyGen(params):
  x_{i,j} ← {0,1}^λ for i ∈ [1,k], j ∈ {0,1}
  sk := (x_{1,0}, x_{1,1}, ..., x_{k,0}, x_{k,1})
  pk := (H.Eval(params, x_{1,0}), H.Eval(params, x_{1,1}), ...,
         H.Eval(params, x_{k,0}), H.Eval(params, x_{k,1}))
  return (sk, pk)

Sign(params, sk, m):
  // Sign must be called at most once per sk.
  (m_1, ..., m_k) := m
  (sk_{1,0}, sk_{1,1}), ..., (sk_{k,0}, sk_{k,1}) := sk
  σ := (sk_{1,m_1}, ..., sk_{k,m_k})
  return σ

Verify(params, pk, m, σ):
  (m_1, ..., m_k) := m
  (pk_{1,0}, pk_{1,1}), ..., (pk_{k,0}, pk_{k,1}) := pk
  return ⋀_{i=1}^k H.Eval(params, σ_i) = pk_{i,m_i}
```

:::

:::{note} Lamport Signature Scheme
:label: def-lamport

Let $\mathsf{H}$ be a hash function and $k$ be a positive integer.
The Lamport one-time signature scheme $\mathsf{LS}[\mathsf{H}, k]$ with message space $\mathcal{M} = \{0,1\}^k$ is defined as above.

:::

:::{tip} Correctness of Lamport Signatures
:label: prop-lamport-correctness

The Lamport signature scheme $\mathsf{LS}[\mathsf{H}, k]$ is correct.

:::

The proof of the correctness is left as an exercise.

:::{attention} Security of Lamport Signatures
:label: thm-ots-lamport-euf-cma

Let $\mathsf{H}$ be a preimage-resistant hash function.
Then the Lamport one-time signature scheme $\mathsf{LS}[\mathsf{H}, k]$ is EUF-CMA secure.
More precisely, for any PPT adversary $\mathcal{A}$ against the EUF-CMA security of $\mathsf{LS}[\mathsf{H, k}]$,
there exists a PPT adversary $\mathcal{B}$ against the preimage resistance of $\mathsf{H}$ such that

$$
\mathcal{A}antage{EUF-CMA}{\mathcal{A}, \mathsf{LS}[\mathsf{H}, k]} \leq 2k\mathcal{A}antage{Preimage}{\mathcal{B}, \mathsf{H}}.
$$

:::

:::{admonition} Sequence of games for the proof that Lamport signatures are EUF-CMA secure
:class: dropdown

**Game EUF-CMA(LS[H,k]):**
```
params ← Setup(1^λ)
(sk, pk) ← KeyGen(params)
Q := ∅
(m*, σ) ← A^Sign(params, pk)
return m* ∉ Q ∧ Verify(params, pk, m*, σ) = 1

Oracle Sign(m):
  assert Q = ∅
  Q := {m}
  return Sign(params, sk, m)
```

**Game G_1:**
```
params ← Setup(1^λ)
(sk, pk) ← KeyGen(params)
[i ← [1,k] ; j ← {0,1}]  // highlighted change
Q := ∅
(m*, σ) ← A^Sign(params, pk)
[if Q = ∅ then]  // highlighted change
[  m ← {0,1}^k]  // highlighted change
[  Sign(m)]  // highlighted change
return m* ∉ Q ∧ Verify(params, pk, m*, σ) = 1

Oracle Sign(m):
  assert Q = ∅
  [assert m_i ≠ j]  // highlighted change
  Q := {m}
  return Sign(params, sk, m)
```

**Game G_2:**
```
params ← Setup(1^λ)
(sk, pk) ← KeyGen(params)
i ← [1,k]
j ← [0,1]
Q := ∅
(m*, σ) ← A^Sign(params, pk)
if Q = ∅ then
  m ← {0,1}^k
  Sign(m)
[assert m*_i = j]  // highlighted change
return m* ∉ Q ∧ Verify(params, pk, m*, σ) = 1

Oracle Sign(m):
  assert Q = ∅
  assert m_i ≠ j
  Q := {m}
  return Sign(params, sk, m)
```

Differences are highlighted.

:::

:::{danger} PROOF
:class: dropdown

The figure above shows a sequence of games for the proof that Lamport signatures are EUF-CMA secure.
The first game is identical to the $\mathsf{Game}~\mathsf{EUF\text{-}CMA}$ game.
Each subsequent game introduces a small change compared to the previous game.
Then we can construct a PPT algorithm that runs a $\mathsf{Game}~\mathsf{EUF\text{-}CMA}$ adversary $\mathcal{A}$ internally and wins $\mathsf{Game}~\mathsf{Preimage}_{\mathsf{H}}$ with the same probability as $\mathcal{A}$ winning the last $\mathsf{Game}~\mathsf{G_2}$.
The rest of the proof is left as an exercise.

:::

## Exercises

:::{exercise}
:label: ex-euf-cma-meaning

What is "existential unforgeability" in "EUF-CMA" and what is "chosen message attack"?

:::

:::{solution} ex-euf-cma-meaning
:class: dropdown

**Existential unforgeability:** The adversary wins if it can forge a signature for **any** message of its choice.

**Chosen message attack:** The adversary has access to a signing oracle and can obtain a signature for a message of its choice before attempting the forgery.

:::

:::{exercise}
:label: ex-signing-oracle-necessity

To see why the security definition for signatures includes a signing oracle, consider $\mathsf{Game}~\mathsf{G}_{\mathsf{OTS}}$ which is the same as $\mathsf{Game}~\mathsf{EUF-CMA}_{\mathsf{OTS}}$ except that the adversary $\mathcal{A}$ is not given access to the signing oracle.
Give an example of a one-time signature scheme that is correct and achieves $\Pr[\mathsf{Game}~\mathsf{G}_{\mathsf{OTS}} = 1] = \mathrm{negl}$ for all PPT adversaries $\mathcal{A}$ but is not EUF-CMA secure.

:::

:::{solution} ex-signing-oracle-necessity
:class: dropdown

Consider the following scheme:

- $\mathsf{KeyGen}(\mathit{pp}) = (\mathit{sk} = x, \mathit{pk} = \mathsf{H.Eval}(\mathit{pp}, x))$ where $x \leftarrow \{0,1\}^\lambda$.
- $\mathsf{Sign}(\mathit{pp}, \mathit{sk}, m) = \mathit{sk}$ (simply outputs the secret key).
- $\mathsf{Verify}(\mathit{pp}, \mathit{pk}, m, \sigma) = 1$ if and only if $\mathsf{H.Eval}(\mathit{pp}, \sigma) = \mathit{pk}$.

This scheme is correct and would be secure in $\mathsf{Game}~\mathsf{G}$ (without signing oracle) since the adversary cannot forge without knowing $\mathit{sk}$, which requires solving the preimage problem.
However, this scheme is obviously insecure in practice: anyone who sees one signature learns the secret key and can forge signatures for any message!
The $\mathsf{Game}~\mathsf{EUF\text{-}CMA}$ game with signing oracle correctly captures this insecurity.

:::

:::{exercise}
:label: ex-lamport-valid-signature

Is $(\mathit{sk}_{1,0}, \mathit{sk}_{2,0}, \mathit{sk}_{3,1})$ a valid signature for the message $m = (0, 0, 1)$ under the Lamport signature scheme $\mathsf{LS}[\mathsf{H}, 3]$?

:::

:::{solution} ex-lamport-valid-signature
:class: dropdown

Yes, it is a valid signature.
For message $m = (0, 0, 1)$ with bits $m_1 = 0$, $m_2 = 0$, $m_3 = 1$, the Lamport signing algorithm outputs $(\mathit{sk}_{1,m_1}, \mathit{sk}_{2,m_2}, \mathit{sk}_{3,m_3}) = (\mathit{sk}_{1,0}, \mathit{sk}_{2,0}, \mathit{sk}_{3,1})$.
The verification will check that $\mathsf{H.Eval}(\mathit{pp}, \mathit{sk}_{i,m_i}) = \mathit{pk}_{i,m_i}$ for each $i \in \{1,2,3\}$, which holds by construction of the public key during key generation.

:::

:::{exercise}
:label: ex-lamport-correctness

Argue that the Lamport signature scheme $\mathsf{LS}[\mathsf{H}, k]$ is correct.

:::

:::{solution} ex-lamport-correctness
:class: dropdown

We prove that the Lamport signature scheme satisfies the correctness definition.

Let $\lambda \in \mathbb{N}$, $\mathit{pp} \in \mathsf{Setup}(1^\lambda)$, $(\mathit{sk}, \mathit{pk}) \in \mathsf{KeyGen}(\mathit{pp})$, and $m = (m_1, \ldots, m_k) \in \{0,1\}^k$.
We need to show that $\Pr[\mathsf{Verify}(\mathit{pp}, \mathit{pk}, m, \mathsf{Sign}(\mathit{pp}, \mathit{sk}, m)) = 1] = 1$.

By the definition of the Lamport scheme:

- $\mathsf{KeyGen}$ sets $\mathit{sk} = (x_{1,0}, x_{1,1}, \ldots, x_{k,0}, x_{k,1})$ and $\mathit{pk} = (\mathit{pk}_{1,0}, \mathit{pk}_{1,1}, \ldots, \mathit{pk}_{k,0}, \mathit{pk}_{k,1})$ where $\mathit{pk}_{i,b} = \mathsf{H.Eval}(\mathit{pp}, x_{i,b})$.
- $\mathsf{Sign}(\mathit{pp}, \mathit{sk}, m)$ outputs $\sigma = (\mathit{sk}_{1,m_1}, \ldots, \mathit{sk}_{k,m_k})$.
- $\mathsf{Verify}(\mathit{pp}, \mathit{pk}, m, \sigma)$ outputs 1 if and only if $\bigwedge_{i=1}^k \mathsf{H.Eval}(\mathit{pp}, \sigma_i) = \mathit{pk}_{i,m_i}$.

For the signature $\sigma$ produced by $\mathsf{Sign}$, we have $\sigma_i = \mathit{sk}_{i,m_i} = x_{i,m_i}$ for each $i \in [1,k]$.
Therefore:

$$
\mathsf{H.Eval}(\mathit{pp}, \sigma_i) = \mathsf{H.Eval}(\mathit{pp}, x_{i,m_i}) = \mathit{pk}_{i,m_i}
$$

holds for all $i \in [1,k]$.
Thus $\mathsf{Verify}(\mathit{pp}, \mathit{pk}, m, \sigma) = 1$ with probability 1 (the Lamport signing algorithm is deterministic).

:::

:::{exercise}
:label: ex-ots-lamport-euf-cma

Complete the proof of the security theorem:

1. Express $\Pr[\mathsf{Game}~\mathsf{G_1}_{} = 1]$ in terms of $\Pr[\mathsf{Game}~\mathsf{EUF-CMA}_{\mathsf{LS[\mathsf{H}, k]}} = 1]$.
2. Express $\Pr[\mathsf{Game}~\mathsf{G_2}_{} = 1]$ in terms of $\Pr[\mathsf{Game}~\mathsf{G_1}_{} = 1]$.
3. Give a PPT algorithm $\mathcal{B}^\mathcal{A}$ that wins $\mathsf{Game}~\mathsf{Preimage}_{\mathsf{H}}$ with probability $\Pr[\mathsf{Game}~\mathsf{G_2}_{} = 1]$.
4. Put it all together to prove the theorem.

:::

:::{solution} ex-ots-lamport-euf-cma
:class: dropdown

We have

$$
\begin{align*}
\Pr[\mathsf{Game}~\mathsf{G_1}_{} = 1] &= \Pr[(\mathsf{Game}~\mathsf{EUF-CMA}_{\mathsf{LS[\mathsf{H}, k]}} = 1) \wedge (m_i \neq j)] \\
&= \Pr[\mathsf{Game}~\mathsf{EUF-CMA}_{\mathsf{LS[\mathsf{H}, k]}} = 1 | m_i \neq j] \Pr[m_i \neq j] \\
&= \frac{1}{2}\Pr[\mathsf{Game}~\mathsf{EUF-CMA}_{\mathsf{LS[\mathsf{H}, k]}} = 1]
\end{align*}
$$

$\mathsf{Game}~\mathsf{G_2}_{} = 1$ only when $m^* \neq m$ and $m^*_i = j$.
Therefore,

$$
\begin{align*}
\Pr[\mathsf{Game}~\mathsf{G_2}_{} = 1] &= \Pr[\mathsf{Game}~\mathsf{G_1}_{} = 1 \wedge m^*_i = j] \\
&= \Pr[m^*_i = j | \mathsf{Game}~\mathsf{G_1}_{} = 1] \Pr[\mathsf{Game}~\mathsf{G_1}_{} = 1] \\
&\geq \frac{1}{k}\Pr[\mathsf{Game}~\mathsf{G_1}_{} = 1]
\end{align*}
$$

The last inequality holds because when $\mathsf{Game}~\mathsf{G_1}$ succeeds, we have $m \neq m^*$ (at least one position differs), $m_i \neq j$, and $i$ is uniformly random. The probability that position $i$ is one where $m$ and $m^*$ differ is at least $1/k$. When they differ at position $i$ and $m_i \neq j$, then $m^*_i = j$ (since the message space is binary).

$\mathcal{B}^\mathcal{A}$ is the same as $\mathsf{Game}~\mathsf{G_2}$ except that

- it gets a preimage challenge $y$ from a challenger and
- replaces $\mathit{pk}_{i, j}$ with $y$
- it returns $\sigma_i$.

Since $m_i \neq j$, the inputs to the adversary (including the signing oracle call) has the same distribution as in $\mathsf{Game}~\mathsf{G_2}$.
Moreover, if $\mathsf{Game}~\mathsf{G_2}_{}$ returns 1, then $\mathsf{H.Eval}(\mathit{pp}, \sigma_i) = \mathit{pk}_{i, j}$
Therefore,

$$
\mathcal{A}antage{Preimage}{\mathcal{B}^\mathcal{A}, \mathsf{H}} = \Pr[\mathsf{Game}~\mathsf{G_2}_{} = 1]
$$

and then combining the inequalities:

$$
\begin{align*}
\mathcal{A}antage{EUF-CMA}{\mathcal{A}, \mathsf{LS}[\mathsf{H, k}]} &= \Pr[\mathsf{Game}~\mathsf{EUF-CMA}_{\mathsf{LS[\mathsf{H}, k]}} = 1] \\
&= 2 \cdot \Pr[\mathsf{Game}~\mathsf{G_1}_{} = 1] \\
&\leq 2k \cdot \Pr[\mathsf{Game}~\mathsf{G_2}_{} = 1] \\
&= 2k \cdot \mathcal{A}antage{Preimage}{\mathcal{B}^\mathcal{A}, \mathsf{H}}
\end{align*}
$$

:::

:::{exercise}
:label: ex-two-queries

Let $\mathsf{Game}~\mathsf{G'}$ be the same as $\mathsf{Game}~\mathsf{EUF\text{-}CMA}$ except that the signing oracle can be called twice.
Where does the proof break?

:::

:::{solution} ex-two-queries
:class: dropdown

The proof breaks at $\mathsf{Game}~\mathsf{G_1}$, which asserts that $m_i \neq j$.
With two signing queries, an adversary can query messages that cover both possible values at every position.
For example, the adversary can query $m^{(1)} = (0, 0, \ldots, 0)$ and $m^{(2)} = (1, 1, \ldots, 1)$.
For any choice of $i \in [1,k]$ and $j \in \{0,1\}$, either $m^{(1)}_i = j$ or $m^{(2)}_i = j$, causing $\mathsf{Game}~\mathsf{G_1}$ to abort.
Therefore, $\Pr[\mathsf{Game}~\mathsf{G_1} = 1] = 0$ for this adversary, breaking the reduction.
This shows why Lamport signatures are only secure for one-time use.

:::

:::{exercise}
:label: ex-post-quantum

(Optional) Why are Lamport signatures considered post-quantum secure?

:::

:::{solution} ex-post-quantum
:class: dropdown

Lamport signatures are considered post-quantum secure because their security relies solely on the preimage resistance of the underlying hash function.
Unlike RSA or ECDSA signatures that rely on the hardness of factoring or discrete logarithm problems (which can be solved efficiently by quantum computers using Shor's algorithm), hash function preimage resistance is believed to remain hard even for quantum computers.
Grover's algorithm provides at most a quadratic speedup for finding preimages, which can be compensated by doubling the hash output length.
Therefore, with appropriately chosen parameters, Lamport signatures remain secure against quantum adversaries.

:::

:::{exercise}
:label: ex-accumulator-variant

(Optional) Consider a variant of Lamport signatures that uses an accumulator.
Instead of storing $\mathit{pk} = (\mathit{pk}_{1,0}, \mathit{pk}_{1,1}, \ldots, \mathit{pk}_{k,0}, \mathit{pk}_{k,1})$, we insert all $2k$ values $\{\mathit{pk}_{i,b} : i \in [1,k], b \in \{0,1\}\}$ into an accumulator and set the public key to be the accumulator value.

1. Describe how signing and verification would work.
2. What are the trade-offs compared to standard Lamport signatures?
3. For security, what property of the accumulator is needed? Sketch how you would modify the EUF-CMA proof.

:::

:::{solution} ex-accumulator-variant
:class: dropdown

**1. Signing and Verification:**

- **KeyGen:** Generate Lamport keys as usual: $x_{i,b} \leftarrow \{0,1\}^\lambda$ and $\mathit{pk}_{i,b} = \mathsf{H.Eval}(\mathit{pp}, x_{i,b})$ for $i \in [1,k], b \in \{0,1\}$.
  Insert all $\mathit{pk}_{i,b}$ values into an accumulator and output $\mathit{pk} = v$ (the accumulator value) and keep $\mathit{sk} = ((x_{1,0}, x_{1,1}), \ldots, (x_{k,0}, x_{k,1}), \mathit{state}_{acc})$ where $\mathit{state}_{acc}$ is the accumulator state.
- **Sign:** For message $m = (m_1, \ldots, m_k)$, compute membership proofs
  $$
  \pi_i \leftarrow \mathsf{Acc.ProveMembership}(\mathit{pp}_{acc}, \mathit{state}_{acc}, \mathit{pk}_{i,m_i})
  $$
  for each $i \in [1,k]$.
  Output $\sigma = ((x_{1,m_1}, \pi_1), \ldots, (x_{k,m_k}, \pi_k))$.
- **Verify:** For each $i \in [1,k]$, compute $\mathit{pk}'_{i,m_i} = \mathsf{H.Eval}(\mathit{pp}, x_{i,m_i})$ and check that
  $$
  \mathsf{Acc.VerifyMembership}(\mathit{pp}_{acc}, v, \mathit{pk}'_{i,m_i}, \pi_i) = 1.
  $$
  Output 1 if all checks pass.

**2. Trade-offs:**

- **Advantage:** Smaller public key (just the accumulator value) instead of $2k$ hash values.
- **Disadvantage:** Larger signatures (must include $k$ membership proofs).

**3. Security:**

We need the accumulator to be collision-free.

**Theorem:** Let $\mathsf{H}$ be a preimage-resistant hash function and $\mathsf{Acc}$ be a collision-free accumulator.
Then the Lamport signature scheme with accumulated public key is EUF-CMA secure.
More precisely, for any PPT adversary $\mathcal{A}$, there exist PPT adversaries $\mathcal{B}_1$ and $\mathcal{B}_2$ such that:

$$
\mathcal{A}antage{EUF-CMA}{\mathcal{A}, \mathsf{LS\text{-}Acc}} \leq 2k \cdot (\mathcal{A}antage{Preimage}{\mathcal{B}_1, \mathsf{H}} + \mathcal{A}antage{Coll}{\mathcal{B}_2, \mathsf{Acc}})
$$

**Proof sketch:**

We follow the game sequence from the original Lamport proof.
Let $\mathsf{Game}~\mathsf{G_2}$ be the game where we guess $(i^*, j^*)$ and abort unless the adversary queries $m$ with $m_{i^*} \neq j^*$ and forges on $m^*$ with $m^*_{i^*} = j^*$.
As in the original proof, $\Pr[\mathsf{Game}~\mathsf{G_2} = 1] \geq \frac{1}{2k} \cdot \mathcal{A}antage{EUF-CMA}{\mathcal{A}, \mathsf{LS\text{-}Acc}}$.

Now we build a PPT reduction $\mathcal{B}_1$ that simulates $\mathsf{Game}~\mathsf{G_2}$:

- $\mathcal{B}_1$ receives preimage challenge $y$, embeds it as $\mathit{pk}_{i^*,j^*} = y$, and runs $\mathsf{Game}~\mathsf{G_2}$ with the adversary.
- When the adversary succeeds in $\mathsf{Game}~\mathsf{G_2}$, they provide $(x^*_{i^*,j^*}, \pi^*_{i^*})$ where the membership proof verifies for some value $\mathit{pk}' = \mathsf{H.Eval}(\mathit{pp}, x^*_{i^*,j^*})$.
- If $\mathit{pk}' = y$, then $\mathcal{B}_1$ outputs $x^*_{i^*,j^*}$ as the preimage of $y$.

However, the adversary might succeed with $\mathit{pk}' \neq y$.
This means the adversary found a valid membership proof for a value different from what we inserted at position $(i^*, j^*)$.
To handle this, we build a PPT reduction $\mathcal{B}_2$:

- $\mathcal{B}_2$ also simulates $\mathsf{Game}~\mathsf{G_2}$ but generates all keys honestly (no preimage challenge).
- When the adversary succeeds with some $(x^*_{i^*,j^*}, \pi^*_{i^*})$ where $\mathit{pk}' = \mathsf{H.Eval}(\mathit{pp}, x^*_{i^*,j^*}) \neq \mathit{pk}_{i^*,j^*}$, this constitutes a collision for the accumulator.
- $\mathcal{B}_2$ outputs this as evidence of breaking collision-freeness.

We have:

$$
\begin{align*}
\Pr[\mathsf{Game}~\mathsf{G_2} = 1] &= \Pr[\mathsf{Game}~\mathsf{G_2} = 1 \wedge \mathit{pk}' = \mathit{pk}_{i^*,j^*}] + \Pr[\mathsf{Game}~\mathsf{G_2} = 1 \wedge \mathit{pk}' \neq \mathit{pk}_{i^*,j^*}]\\
&\leq \mathcal{A}antage{Preimage}{\mathcal{B}_1, \mathsf{H}} + \mathcal{A}antage{Coll}{\mathcal{B}_2, \mathsf{Acc}}
\end{align*}
$$

Since $\mathcal{A}antage{EUF-CMA}{\mathcal{A}, \mathsf{LS\text{-}Acc}} \leq 2k \cdot \Pr[\mathsf{Game}~\mathsf{G_2} = 1]$ (from the original proof), we get:

$$
\mathcal{A}antage{EUF-CMA}{\mathcal{A}, \mathsf{LS\text{-}Acc}} \leq 2k \cdot (\mathcal{A}antage{Preimage}{\mathcal{B}_1, \mathsf{H}} + \mathcal{A}antage{Coll}{\mathcal{B}_2, \mathsf{Acc}})
$$

:::

:::{exercise}
:label: ex-stateful-many-time

(Optional) Describe how to build a stateful many-time signature scheme from a one-time signature scheme and an accumulator.
Your construction should support signing up to $n$ messages (where $n$ is fixed at setup).

:::

:::{solution} ex-stateful-many-time
:class: dropdown

Here is a construction for a stateful many-time signature scheme:

**Setup/KeyGen:**

1. Generate $n$ one-time signature key pairs: $(\mathit{sk}_i, \mathit{pk}_i) \leftarrow \mathsf{OTS.KeyGen}(\mathit{pp})$ for $i \in [1,n]$.
2. Insert all $n$ public keys $\{\mathit{pk}_1, \ldots, \mathit{pk}_n\}$ into an accumulator.
3. The master public key is $\mathit{pk} = v$ (the accumulator value).
4. The master secret key is $\mathit{sk} = ((\mathit{sk}_1, \mathit{pk}_1), \ldots, (\mathit{sk}_n, \mathit{pk}_n), \mathit{state}_{acc})$ where $\mathit{state}_{acc}$ is the accumulator state.
5. Initialize a counter $\mathit{ctr} = 0$.

**Sign (stateful):**

1. Increment $\mathit{ctr} \leftarrow \mathit{ctr} + 1$.
2. If $\mathit{ctr} > n$, abort (all keys have been used).
3. Sign the message using the $\mathit{ctr}$-th OTS key: $\sigma_{ots} \leftarrow \mathsf{OTS.Sign}(\mathit{pp}, \mathit{sk}_{\mathit{ctr}}, m)$.
4. Generate a membership proof: $\pi \leftarrow \mathsf{Acc.ProveMembership}(\mathit{pp}_{acc}, \mathit{state}_{acc}, \mathit{pk}_{\mathit{ctr}})$.
5. Output $\sigma = (\mathit{ctr}, \mathit{pk}_{\mathit{ctr}}, \sigma_{ots}, \pi)$.

**Verify:**

1. Parse $\sigma = (i, \mathit{pk}_i, \sigma_{ots}, \pi)$.
2. Verify the OTS signature: check that $\mathsf{OTS.Verify}(\mathit{pp}, \mathit{pk}_i, m, \sigma_{ots}) = 1$.
3. Verify accumulator membership: check that $\mathsf{Acc.VerifyMembership}(\mathit{pp}_{acc}, v, \mathit{pk}_i, \pi) = 1$.
4. Output 1 if both checks pass, 0 otherwise.

The scheme is stateful because the signer must track which keys have been used (via $\mathit{ctr}$) to ensure no OTS key is reused.
Security relies on the one-time unforgeability of the OTS scheme and the collision-freeness of the accumulator.

:::

:::{exercise}
:label: ex-winternitz

(Optional) Read Sections 2.2, Theorem 1, and 3.2 of Hülsing (AFRICACRYPT 2013) about the Winternitz One-Time Signature scheme.
Explain what challenges the security reduction embeds and how they are embedded.

:::